In [18]:
import sys
sys.path.insert(0, "/Users/aimachine/PyImage/utils")
import numpy as np
import tensorflow as tf
import importlib
import matplotlib.pyplot as plt
import os
import glob
import skimage
import cv2
from skimage import measure
from skimage import filters
from sklearn.cluster import DBSCAN
from skimage.filters import threshold_otsu
from tifffile import imread
import scipy
from sklearn.feature_extraction import image as imp
from sklearn.cluster import spectral_clustering
from scipy import ndimage
import time
from Normalize import normalizeFloat, normalizeMinMax, Path, save_tiff_imagej_compatible

In [12]:
basedir = '/Users/aimachine/Documents/JuliaData/Julia/Subset/'
targetdir = '/Users/aimachine/Documents/JuliaData/Julia/SpectralSegmentedData/'

In [13]:
Path = os.path.join(basedir, '*.tif')
X = []
Names = []
filesRaw = glob.glob(Path)
maxtime = 0
axes = 'TYX'
for fname in filesRaw:
      x = imread(fname)
      min = np.amin(x)
      max = np.amax(x)
      x = normalizeMinMax(x, min, max)  
      X.append(x)
    
     
      Names.append(fname)
      if X[0].shape[0] > maxtime:
        maxtime = X[0].shape[0]
X.sort
Names.sort


ReshapeX = []
for i in range(len(X)):
    y = np.zeros((maxtime, X[0].shape[1], X[0].shape[2]))
    image = X[i]
    
    y[:X[i].shape[0],:,:] = image[:,:,:]
    ReshapeX.append(y)
    
ReshapeX.sort

print('Total number of input images = ', len(X))
print('Image size = ', X[0].shape)
print('Time dim = ', maxtime)

timerange = maxtime



Total number of input images =  1
Image size =  (2, 254, 256)
Time dim =  2


In [28]:
def Clustering(img):
    Labels = img
    for j in range(timerange):
      smallimg = img[j,:,:]
      
      """Put clustering code"""
      graph = imp.img_to_graph(smallimg)
      beta = 5
      eps = 1e-6
      graph.data = np.exp(-beta * graph.data / smallimg.std()) + eps 
      N_REGIONS = 28
    
    t0 = time.time()
    labels = spectral_clustering(graph, n_clusters=N_REGIONS,
                                 assign_labels='discretize',
                                 random_state=1)
    t1 = time.time()
    labels = labels.reshape(smallimg.shape)
    print('time taken', t1 - t0)
    plt.figure(figsize=(5, 5))
    plt.imshow(smallimg,   cmap=plt.cm.gray)
    for l in range(N_REGIONS):
        plt.contour(labels == l, contours=1,
                    colors=[plt.cm.nipy_spectral(l / float(N_REGIONS)), ])
       
    
    

In [ ]:
for i in range(0, len(ReshapeX)):
  newimg = ReshapeX[i]
  plt.imshow(newimg[0,:,:])
  Clustering(newimg)
    

